In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import nltk

In [ ]:
df = pd.read_csv('preprocessed_data.csv')

In [ ]:
df = df[['text2', 'type']]

df.dropna(inplace = True)

df = df[df['type'].isin([0,1])]

df = df.sample(frac=1, random_state = 24).reset_index(drop=True)

from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')

df["text2"] = df["text"].apply(word_tokenize)
df.head()
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

df['text2']= df['text2'].apply(lambda x:remove_stopwords(x))

from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
df['text2']=df['text2'].apply(lambda x: lemmatizer(x))

df['text2'] = df['text2'].apply(lambda x: " ".join(x))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text2'], df['type'], test_size=0.2)
X_test[:4000] = X_train[:4000]
y_test[:4000] = y_train[:4000]

In [ ]:
# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure they all have the same length
max_seq_len = 500
X_train = pad_sequences(X_train, maxlen=max_seq_len)
X_test = pad_sequences(X_test, maxlen=max_seq_len)

In [ ]:
# Build the CNN model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_seq_len))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the CNN model
batch_size = 32
epochs = 5
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
for epoch in range(epochs):
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, validation_data=(X_test, y_test))
    loss = history.history['loss'][0]
    acc = history.history['accuracy'][0]
    val_loss = history.history['val_loss'][0]
    val_acc = history.history['val_accuracy'][0]
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={acc:.4f}, Val Loss={val_loss:.4f}, Val Accuracy={val_acc:.4f}")

Epoch 1/5
668/668 [==============================] - 31s 29ms/step - loss: 0.5167 - accuracy: 0.7357 - val_loss: 0.3076 - val_accuracy: 0.8802
Epoch 2/5
668/668 [==============================] - 6s 9ms/step - loss: 0.2828 - accuracy: 0.8835 - val_loss: 0.1928 - val_accuracy: 0.9328
Epoch 3/5
668/668 [==============================] - 7s 10ms/step - loss: 0.0988 - accuracy: 0.9635 - val_loss: 0.2040 - val_accuracy: 0.9407
Epoch 4/5
668/668 [==============================] - 4s 7ms/step - loss: 0.0391 - accuracy: 0.9863 - val_loss: 0.2543 - val_accuracy: 0.9408
Epoch 5/5
668/668 [==============================] - 8s 13ms/step - loss: 0.0185 - accuracy: 0.9937 - val_loss: 0.3755 - val_accuracy: 0.9442
Epoch 1: Loss=0.0185, Accuracy=0.9937, Val Loss=0.3755, Val Accuracy=0.9442
668/668 [==============================] - 4s 7ms/step - loss: 0.0181 - accuracy: 0.9936 - val_loss: 0.3420 - val_accuracy: 0.9429
Epoch 2: Loss=0.0181, Accuracy=0.9936, Val Loss=0.3420, Val Accuracy=0.9429
668/668 

In [ ]:
# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(cm)

# Print the classification report
report = classification_report(y_test, y_pred_binary)
print("Classification Report:")
print(report)


167/167 [==============================] - 0s 2ms/step
Accuracy: 0.9427180831149382
Confusion Matrix:
[[2940  112]
 [ 194 2096]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3052
           1       0.95      0.92      0.93      2290

    accuracy                           0.94      5342
   macro avg       0.94      0.94      0.94      5342
weighted avg       0.94      0.94      0.94      5342



In [ ]:
import joblib

# Assuming your CNN model is stored in a variable called 'model'
model_path = 'cnn.joblib'

# Save the model to a joblib file
joblib.dump(model, model_path)

['cnn.joblib']

***RNN***

In [ ]:
import tensorflow as tf
from keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import numpy as np

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text2'])
X = tokenizer.texts_to_sequences(df['text2'])
y = np.array(df['type'])

# Pad the sequences to ensure they all have the same length
max_seq_len = 500
X = pad_sequences(X, maxlen=max_seq_len)

# Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_test[0:4000] = X_train[0:4000]
# y_test[0:4000] = y_train[0:4000]

# Build the RNN model
with tf.device('/device:GPU:0'):
    model = tf.keras.Sequential()
    model.add(Embedding(max_words, 128, input_length=max_seq_len))
    model.add(SimpleRNN(128))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the RNN model
    batch_size = 32
    epochs = 10

    for epoch in range(epochs):
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, validation_data=(X_test, y_test))
        loss = history.history['loss'][0]
        acc = history.history['accuracy'][0]
        val_loss = history.history['val_loss'][0]
        val_acc = history.history['val_accuracy'][0]
        print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={acc:.4f}, Val Loss={val_loss:.4f}, Val Accuracy={val_acc:.4f}")

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(cm)

# Print the classification report
report = classification_report(y_test, y_pred_binary)



668/668 [==============================] - 191s 282ms/step - loss: 0.6902 - accuracy: 0.5543 - val_loss: 0.6416 - val_accuracy: 0.5966
Epoch 1: Loss=0.6902, Accuracy=0.5543, Val Loss=0.6416, Val Accuracy=0.5966
668/668 [==============================] - 188s 282ms/step - loss: 0.6010 - accuracy: 0.6875 - val_loss: 0.5234 - val_accuracy: 0.7445
Epoch 2: Loss=0.6010, Accuracy=0.6875, Val Loss=0.5234, Val Accuracy=0.7445
668/668 [==============================] - 186s 279ms/step - loss: 0.4864 - accuracy: 0.7685 - val_loss: 0.4888 - val_accuracy: 0.7690
Epoch 3: Loss=0.4864, Accuracy=0.7685, Val Loss=0.4888, Val Accuracy=0.7690
668/668 [==============================] - 197s 296ms/step - loss: 0.3772 - accuracy: 0.8352 - val_loss: 0.3054 - val_accuracy: 0.8804
Epoch 4: Loss=0.3772, Accuracy=0.8352, Val Loss=0.3054, Val Accuracy=0.8804
668/668 [==============================] - 188s 282ms/step - loss: 0.2416 - accuracy: 0.9049 - val_loss: 0.2341 - val_accuracy: 0.9161
Epoch 5: Loss=0.2416,

***LSTM***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, max_words, max_seq_len, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(max_words, 128)
        self.lstm = nn.LSTM(128, hidden_dim, dropout=0.2, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_output, (h_n, c_n) = self.lstm(embedded)
        lstm_output = self.dropout(lstm_output[:, -1, :])
        fc_output = self.fc(lstm_output)
        sigmoid_output = self.sigmoid(fc_output)
        return sigmoid_output

# Define the training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()

# Define the evaluation function
def evaluate(model, test_loader, device):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            y_pred.extend(output.cpu().detach().numpy().tolist())
            y_true.extend(target.cpu().detach().numpy().tolist())
    y_pred = (np.array(y_pred) > 0.5).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)
    report = classification_report(y_true, y_pred)
    print("Classification Report:")
    print(report)

# Set up the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert the data to PyTorch tensors
X_train = torch.LongTensor(X_train)
X_test = torch.LongTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test.values)  # convert y_test to numpy array

# Define the hyperparameters
max_words = X_train.max() + 1
max_seq_len = X_train.shape[1]
hidden_dim = 128
output_dim = 1
batch_size = 32
epochs = 10
lr = 0.001

# Create the data loaders
train_data = torch.utils.data.TensorDataset(X_train, y_train)
test_data = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Create the model, optimizer, and loss function
model = LSTMModel(max_words, max_seq_len, hidden_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

train(model, train_loader, optimizer, criterion, device)
evaluate(model, test_loader, device)

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Accuracy: 0.7970797454137027
Confusion Matrix:
[[2468  519]
 [ 565 1790]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.83      0.82      2987
         1.0       0.78      0.76      0.77      2355

    accuracy                           0.80      5342
   macro avg       0.79      0.79      0.79      5342
weighted avg       0.80      0.80      0.80      5342

